# Black-Scholes: Option pricing with Fourier transform

## Mathematics


In order to use the Fourier method for option pricing, we need the characteristic function of the process we consider and the characteristic function of the payoff.

The characteristic function is available analytically for all Lévy processes (including Brownian motion which is a special case) even when the PDF is not. We consider a log-normal process such that

\begin{equation}
    \log \frac{S_{t + \Delta t}}{S_t} \sim \mathcal{N} \left( \Big(r - q - \frac{1}{2} \sigma^2 \Big)\Delta t, \sigma^2 \Delta t \right)
\end{equation}

where $S_t$ is the stock price at time $t$, $\sigma$ is the stock price volatility, $r$ is the risk-free interest rate and $q$ is the dividend rate.

The characteristic function is given by 

\begin{equation}
    \psi(\xi) = \exp \left[ i \xi \left(r - q - \frac{1}{2} \sigma^2 \right) \Delta t - \frac{1}{2} \xi^2 \sigma^2 \Delta t \right]
\end{equation}

where $i$ is the imaginary unit such that $i = \sqrt{-1}$.

The characteristic function of the payoff is also available analytically. The payoff of a vanilla European option, $g$, is defined as

\begin{equation}
    g(S_T) = \max \big(\theta(S_T - K), 0 \big) = \big(\theta(S_T - K) \big)^+
\end{equation}

where $S_T$ is the stock price at maturity $T$, $K$ is the strike price and $\theta$ is equal to 1 for a call and -1 for a put. 

Because the payoff function diverges in the limits (i.e. $g(S_T) \to \infty$ as $S_T \to \infty$ for a call), we introduce a damping factor $e^{\alpha S_T}$ with $\alpha \in \mathbb{R}$. Multiplying the payoff by this factor, with $\alpha < 0$ for a call and $\alpha > 0$ for a put, makes the function square integrable. 

The payoff is expressed as a function of the log-price 

\begin{equation}
    x = \log \frac{S_T}{S_0},
\end{equation}

the log-strike

\begin{equation}
    k = \log \frac{K}{S_0},
\end{equation}

and (potentially) the upper log-barrier

\begin{equation}
    u = \log \frac{U}{S_0},
\end{equation}

and the lower log-barrier

\begin{equation}
    l = \log \frac{L}{S_0}.
\end{equation}

Our damped payoff function becomes

\begin{equation}
    g(x) = e^{\alpha x} S_0 \big(\theta (e^x - e^k) \big)^+ \mathbb{1}_{[l, u]}(x)
\end{equation}

where $l$ and $u$ are set equal to the upper and lower truncation limits for vanilla options. Also, note that the damping factor is slightly changed (i.e. $e^{\alpha x} \neq e^{\alpha S_T}$) but this is not a problem because damping is just a technical trick which is undone at the end.

The Fourier transform of the damped payoff is given by

\begin{align}
    \hat{g}(\xi) = \mathscr{F}_{x \to \xi} [g(x)] &= \int_{\mathbb{R}} e^{i \xi x} e^{\alpha x} S_0 \big(\theta (e^x - e^k) \big)^+ \mathbb{1}_{[l, u]}(x) dx \notag \\
    &= S_0 \int_l^u e^{(i \xi + \alpha)x} \big(\theta (e^x - e^k) \big)^+ dx \notag \\
    &= S_0 \int_{\nu}^{\eta} e^{(i \xi + \alpha)x} \big(e^x - e^k \big) dx \notag \\
    &= S_0 \int_{\nu}^{\eta} e^{(1 + i \xi + \alpha)x} dx - S_0 \int_{\nu}^{\eta} e^{k + (i \xi + \alpha)x} dx \notag \\
    &= S_0 \left( \frac{e^{(1 + i\xi + \alpha)\eta} - e^{(1 + i\xi + \alpha)\nu}}{1 + i\xi + \alpha} - \frac{e^{k + (i\xi + \alpha)\eta} - e^{k + (i\xi + \alpha)\nu}}{i\xi + \alpha} \right)
\end{align}

where

\begin{equation}
    \eta = \begin{cases} 
        u & \text{for a call}  \\
        l & \text{for a put} 
    \end{cases},
\end{equation}

and 

\begin{equation}
    \nu = \begin{cases} 
        \max(k, l) & \text{for a call}  \\
        \min(k, u) & \text{for a put} 
    \end{cases}.
\end{equation}

In order to find the option value, $V$, we need to discount the expected payoff

\begin{align}
    V &= e^{-rT} \mathbb{E} \left[ g(X_T) e^{- \alpha X_T} \, \middle| \; X_0 = 0 \right] \notag \\
      &= e^{-rT} \int_{\mathbb{R}} g(X_T) e^{- \alpha X_T} f_X(x, T) dx
\end{align}

where $e^{-\alpha X_T}$ is the undamping factor and $f$ is the PDF of $X_t$.

Thanks to the [Parseval-Plancherel theorem](#plancherel), this integral can be computed in Fourier space. Hence, we have

\begin{equation}
    V = \frac{e^{-rT}}{2 \pi} \int_{\mathbb{R}} \hat{g}(\xi) \psi^*(\xi + i \alpha, T) d\xi
\end{equation}

where $\psi(\xi, t) = \hat{f}_X(x, t)$ is the characteristic function of $X_t$ and * denotes the complex conjugate. Note that we can include the undamping factor in $\psi^*(\xi + i \alpha, T)$ thanks to the [Shift theorem](#shift).

## Implementation

First, let's import some useful libraries.

In [1]:
import numpy as np
from scipy.interpolate import interp1d

Let's implement a function which returns the damped payoff and its Fourier transform.

In [2]:
# Function for the Fourier transform of the payoff
def payoff(x, xi, alpha, K, L, U, C, call=1):
    
    # scale
    S = C*np.exp(x)   
    
    # payoff
    if call == 1: # call
        g = np.exp(alpha*x) * np.maximum(S - K, 0) * (S>=L).astype(int) * (S<=U).astype(int)
    else: # put
        g = np.exp(alpha*x) * np.maximum(K - S, 0) * (S>=L).astype(int) * (S<=U).astype(int)
        
    # Analitical Fourier transform of the payoff
    l = np.log(L/C) # lower log barrier
    k = np.log(K/C) # log strike
    u = np.log(U/C) # upper log barrier
    
    # Integration bounds
    if call == 1:
        a = max(l, k)
        b = u
    else:
        a = min(k, u)
        b = l
        
    xi_2 = alpha + 1j*xi
    
    # Fourier transform of damped payoff
    with np.errstate(divide='ignore', invalid='ignore'): # disable warning for when alpha = 0
        G = C*((np.exp(b * (1 + xi_2)) - np.exp(a * (1 + xi_2))) / (1 + xi_2) \
           - (np.exp(k + b*xi_2) - np.exp(k + a*xi_2)) / xi_2)
    
    # Eliminable discontinuities for xi = 0, otherwise 0/0 = NaN
    if alpha == 0:
        G[int(np.floor(len(G)/2))] = C*(np.exp(b)-np.exp(a)-np.exp(k)*(b-a))
    elif alpha == -1:
        G[int(np.floor(len(G)/2))] = C*(b-a+np.exp(k-b)-np.exp(k-a))
    
    return g, G, S

Now, let's create a function which returns the call and put options prices using the Fourier transform method.

In [3]:
def fourier_pricing(S0, K, T, r, q, sigma, alpha, xwidth, ngrid):
    
    # Risk-neutral measure
    muRN = r-q-0.5*sigma**2 # drift

    # Fourier parameters
    xwidth = 6  # width of the support in real space
    ngrid = 2**8 # number of grid points
    alpha = -10 # damping factor for a call

    # Grids in real and Fourier space
    N = int(ngrid/2)
    b = xwidth/2 # upper bound of the support in real space
    dx = xwidth/ngrid
    x = dx * np.linspace(-N, N-1, 2*N)
    dxi = 2*np.pi/xwidth # Nyquist relation
    xi = dxi * np.linspace(-N, N-1, 2*N)

    # Characteristic function at time T for a call
    xia = xi + 1j*alpha 
    psi = 1j*muRN*xia -0.5*(sigma*xia)**2 # characteristic exponent
    psi_c = np.exp(psi*T) # characteristic function
    
    # Characteristic function at time T for a put
    xia = xi - 1j*alpha # put
    psi = 1j*muRN*xia -0.5*(sigma*xia)**2 # characteristic exponent
    psi_p = np.exp(psi*T) # characteristic function
    
    # Fourier transform of the payoff
    U = S0 * np.exp(b)
    L = S0 * np.exp(-b)
    gc, Gc, S = payoff(x, xi, alpha, K, L, U, S0, 1) # call
    gp, Gp, S = payoff(x, xi, -alpha, K, L, U, S0, 0) # put

    # Discounted expected payoff computed with the Plancherel theorem    
    c = np.exp(-r*T)*np.real(np.fft.fftshift(np.fft.fft(np.fft.ifftshift(Gc*np.conj(psi_c)))))/xwidth        
    Vc = interp1d(S, c, kind='slinear')(S0) # Call value
    p = np.exp(-r*T)*np.real(np.fft.fftshift(np.fft.fft(np.fft.ifftshift(Gp*np.conj(psi_p)))))/xwidth 
    Vp = interp1d(S, p, kind='slinear')(S0) # Put value
    
    return Vc, Vp

Let's try the function with some parameters and print the prices of the call and the put.

In [4]:
# Market parameters
T = 1      # maturity
S0 = 1     # spot price
K = 1.1    # strike price
r = 0.05   # risk-free interest rate
q = 0.02   # dividend rate

# Model parameter
sigma = 0.4 # volatility

# Fourier parameters
xwidth = 6   # width of the support in real space
ngrid = 2**8 # number of grid points
alpha = -10  # damping factor for a call

Vc, Vp = fourier_pricing(S0, K, T, r, q, sigma, alpha, xwidth, ngrid)

print('The value of the call is ' + str(np.round(Vc, 5)) + '.')
print('The value of the put is  ' + str(np.round(Vp, 5)) + '.')


The value of the call is 0.12965.
The value of the put is  0.19581.


## Theorems

### Parseval-Plancherel theorem

<a name="plancherel"></a>

The Parseval-Plancherel theorem states that

\begin{equation}
    \int_{\mathbb{R}} f(x)g^*(x) dx = \frac{1}{2 \pi} \int_{\mathbb{R}} \hat{f}(\xi)\hat{g}^*(\xi) d\xi.
\end{equation}

To prove the statement, we start by substituting in the LHS. We have

\begin{equation}
    f(x) = \mathscr{F}_{\xi \to x}^{-1} [\hat{f}(\xi)] = \frac{1}{2 \pi} \int_{\mathbb{R}} e^{-i x \xi} \hat{f}(\xi) d\xi,
\end{equation}

\begin{equation}
    g(x) = \mathscr{F}_{\xi \to x}^{-1} [\hat{g}(\xi)] = \frac{1}{2 \pi} \int_{\mathbb{R}} e^{-i x \xi} \hat{g}(\xi) d\xi.
\end{equation}

Using the fact that $(ab)^* = a^*b^*$, $(e^{-i x \xi})^* = e^{+i x \xi}$ and we have

\begin{equation}
    g^*(x) = \frac{1}{2 \pi} \int_{\mathbb{R}} e^{i x \xi} \hat{g}^*(\xi) d\xi.
\end{equation}

Therefore,

\begin{align}
    \int_{\mathbb{R}} f(x)g^*(x) dx &= \frac{1}{(2 \pi)^2} \int_{\mathbb{R}} \left( \int_{\mathbb{R}} e^{-i x \xi} \hat{f}(\xi) d\xi \right) \left( \int_{\mathbb{R}} e^{i x \xi'} \hat{g}^*(\xi') d\xi' \right) dx \\
    & = \frac{1}{2 \pi} \int_{\mathbb{R}} \int_{\mathbb{R}} \hat{f}(\xi) \hat{g}^*(\xi') \frac{1}{2 \pi} \int_{\mathbb{R}} e^{-i (\xi - \xi') x} dx d\xi' d\xi.
\end{align}

Because

\begin{equation}
    \frac{1}{2 \pi} \int_{\mathbb{R}} e^{-i (\xi - \xi') x} dx = \delta(\xi - \xi')
\end{equation}

where $\delta(.)$ is Dirac's delta function, we have

\begin{equation}
    \int_{\mathbb{R}} f(x)g^*(x) dx = \frac{1}{2 \pi} \int_{\mathbb{R}} \hat{f}(\xi) \int_{\mathbb{R}} \hat{g}^*(\xi') \delta(\xi - \xi') d\xi' d\xi.
\end{equation}

Finally, because 

\begin{equation}
    \int_{\mathbb{R}} \hat{g}^*(\xi') \delta(\xi - \xi') d\xi' = \hat{g}^*(\xi),
\end{equation}

we obtain 

\begin{equation}
    \int_{\mathbb{R}} f(x)g^*(x) dx = \frac{1}{2 \pi} \int_{\mathbb{R}} \hat{f}(\xi)\hat{g}^*(\xi) d\xi.
\end{equation}

### Shift theorem

<a name="shift"></a>

The shift theorem states that 

\begin{equation}
    \mathscr{F}_{x \to \xi} [f(x)e^{- \alpha x}] = \hat{f} (\xi + i \alpha).
\end{equation}

The proof is

\begin{equation}
    \mathscr{F}_{x \to \xi} [f(x)e^{- \alpha x}] = \int_{\mathbb{R}}  e^{i \xi x} e^{- \alpha x} f(x) dx = \int_{\mathbb{R}}  e^{i (\xi + i \alpha) x} f(x) dx = \hat{f} (\xi + i \alpha)
\end{equation}

where we used the fact that $-1 = i^2$.

It can also be proved using the inverse Fourier tranform as

\begin{equation}
    \mathscr{F}_{\xi \to x}^{-1} [\hat{f} (\xi + i \alpha)] = \frac{1}{2 \pi} \int_{\mathbb{R}} e^{-i x \xi} \hat{f} (\xi + i \alpha) d\xi = \frac{e^{- \alpha x}}{2 \pi} \int_{\mathbb{R}} e^{-i x (\xi + i \alpha)} \hat{f} (\xi + i \alpha) d(\xi + i \alpha) = \frac{e^{- \alpha x}}{2 \pi} \int_{\mathbb{R}} e^{-i x \xi'} \hat{f} (\xi') d\xi' = f(x)e^{- \alpha x}
\end{equation}

where we used a change of variable.